In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder

# import keras
from keras.regularizers import l1, l2, l1_l2
from keras.layers import (Dense, Activation, Flatten, Conv2D, 
                          MaxPooling2D,Dropout,BatchNormalization)
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils


Using TensorFlow backend.


In [2]:

TSMC_NEWS = pd.read_csv('/Users/liaoshousan/Desktop/大四上/R語言與資料科學導論/Final_PROJECT-StockPrice Prediction/做好所有預處理的新聞/TSMC_NEWS.csv')
TSMC_PRICE = pd.read_csv('/Users/liaoshousan/Desktop/大四上/R語言與資料科學導論/Final_PROJECT-StockPrice Prediction/做好所有預處理的股價/TSMC_Stock.csv')


前處理

In [3]:
token_list = []
for NEWS in TSMC_NEWS['CONTENT']:
    NEWS = re.sub(r'[。，（）_?\d]','',NEWS)
    #這邊想再加一行過濾亂碼的條件，但是還沒搞懂
    seg_list = jieba.cut(NEWS)
    token = ' '.join(NEWS)
    token_list.append(token)


In [4]:
with open("/Users/liaoshousan/Desktop/大四上/R語言與資料科學導論/Final_PROJECT-StockPrice Prediction/兩千個重要正負面詞彙/token_2000.txt", "r") as f:
    token_2000 = f.read().splitlines()

token_2000 = list(set(token_2000))
count_vec = CountVectorizer(vocabulary=token_2000)
token_matrix = count_vec.fit_transform(token_list)
token_array = pd.DataFrame(token_matrix.toarray(),columns = token_2000)



In [5]:
X = pd.concat([TSMC_NEWS,token_array],axis = 1)
X = X.sort_values(by = 'TIME')
y = X['收盤價']
X = X.drop(columns=['TITLE','TIME','DESCRIPTION','CONTENT','status','漲跌價差','成交筆數','status','收盤價'],axis = 1)



In [6]:
from sklearn.model_selection import train_test_split
X = np.array(X)
X = X.reshape(X.shape[0], 1, X.shape[1])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

建神經網路模型

In [7]:
# define baseline model
def baseline_modelCNN():
    # create model
    model = Sequential()
    model.add(Dense(256, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(128,activation ='relu',
                    kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
# define baseline model
from keras.layers import LSTM
def baseline_model_RNN():
    # create model
    model = Sequential()
    
    model.add(LSTM(
        input_dim=X.shape[2],
        output_dim = 50,
        return_sequences = True))

    model.add(Dropout(0.5))
    model.add(LSTM(
        100,
        return_sequences = False))
    model.add(Dropout(0.5))
    model.add(Dense(
        output_dim = 1))
    model.add(Activation('linear'))
              
    # Compile model
    model.compile(loss = 'mse', optimizer='adam')
    
    return model

In [9]:
model = baseline_model_RNN()
model.summary()
model.fit(X_train,y_train,batch_size = 64,nb_epoch = 50,validation_split = 0.3)

/Users/liaoshousan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  # Remove the CWD from sys.path while we load stuff.
/Users/liaoshousan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(None, 200..., units=50)`
  # Remove the CWD from sys.path while we load stuff.
/Users/liaoshousan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`
/Users/liaoshousan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 50)          410200    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 50)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 470,701
Trainable params: 470,701
Non-trainable params: 0
________________________________________________

In [10]:
model.predict(X_test)
# prediction = LSTM.predict_sequence_multiple(model,X_test,50,50)
# LSTM.plot_results_multiple(prediction,y_test,50)

array([[189.40115],
       [189.40115],
       [189.40115],
       ...,
       [189.40115],
       [189.40115],
       [189.40115]], dtype=float32)

為什麼用神經網路都會預測一樣的數字？

視覺化

In [11]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='center right')
plt.show()


NameError: name 'history' is not defined

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='center right')
plt.show()

### 發現資料筆數差距太大了，讓模型整個爛掉

In [ ]:
import matplotlib
